In [9]:
import model_helper as mh
import torch
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import time
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support
import json
from datetime import datetime
import pandas as pd

In [11]:
def gnn_toposearch(ds_split_seeds, dataset_folder, method, gnn_params, ds_name, results_path, gs_results):
    epochs = 5000
    results = {
        'test_f1':None,
        'test_auc':None,
        'test_precision':None,
        'test_recall':None,
        'lr':None,
        'w':None,
        'K':None,
        'F':None,
        'K1':None,
        'K2':None,
        'F1':None,
        'F2':None,
        'epoch':None
    }
    best_model_diff = 1 #1 is worst
    closest_f1 = 0
    count = 1
    for lr in gnn_params['lr']:
        for w in gnn_params['w']:
            if method == "GINSAGE":
                
                for K1 in gnn_params['K1']:
                    for K2 in gnn_params['K2']:
                        for F1 in gnn_params['F1']:
                            for F2 in gnn_params['F2']:
                                f1_scores_for_group = []
                                best_epochs_for_group = []
                                auc_scores_for_group = []
                                precision_scores_for_group = []
                                recall_scores_for_group = []
                                seed_count = 0
                                for seed in ds_split_seeds:
                                    print(method, count, 'out of', len(gnn_params['lr'])*len(gnn_params['w'])*len(gnn_params['K1'])*len(gnn_params['K2'])*len(gnn_params['F1'])*len(gnn_params['F2'])*len(ds_split_seeds))
                                    
                                    ds = mh.Dataset()
                                    ds.load_dataset(folder=dataset_folder+ds_name,splits=[0.5,0.2,0.3],split_type="normal",split_seed=seed)
                                    gin_feature_indices = [ds.feature_labels.index(item) for item in ['node_deg_out_unique', 'node_deg_in_unique']]
                                    #gin_feature_indices = [ds.feature_labels.index(item) for item in ['node_deg_out', 'node_deg_in']]
                                    #gin_feature_indices = [-1]
                                    
                                    model = mh.Model(ds.data, gridsearch_flag=True)
                                    
                                    additional_params = {'K1':K1, 'K2':K2, 'F1':F1, 'F2':F2, 'gin_feature_indices':gin_feature_indices}
                                    model.w = w
                                    model.lr = lr
                                    model.load_model("GINSAGE",additional_params=additional_params)
                                    model.best_f1_from_gs = gnn_params['mean_f1']
                                    #model.train_model(epochs=gnn_params['epochs'][seed_count])
                                    model.train_model(epochs=5000)
                                    f1_scores_for_group.append(model.gridsearch_results['test_f1'])
                                    best_epochs_for_group.append(model.gridsearch_results['test_epoch'])
                                    auc_scores_for_group.append(model.gridsearch_results['test_auc'])
                                    precision_scores_for_group.append(model.gridsearch_results['test_precision'])
                                    recall_scores_for_group.append(model.gridsearch_results['test_recall'])
                                    count+=1
                                    seed_count+=1
                                '''
                                if np.abs(model.gridsearch_results['test_f1'] - gnn_params['mean_f1']) < best_model_diff:
                                    best_model_diff = np.abs(model.gridsearch_results['test_f1'] - gnn_params['mean_f1'])
                                    closest_f1 = model.gridsearch_results['test_f1']
                                    tpfp = model.check_topology_performance(folder=ds.folder, data=ds.data)
                                    results = {
                                    'tp':tpfp[0],
                                    'fp':tpfp[1],
                                    }
                                '''
                                '''
                                for ele in results:
                                    gs_results[ds_name][method][ele] = results[ele]
                                    print(ele,results[ele])
                                with open(results_path, 'w') as file:
                                    json.dump(gs_results, file, indent=2)
                                '''
                                
            else:
                for K in gnn_params['K']:
                    for F in gnn_params['F']:
                        f1_scores_for_group = []
                        best_epochs_for_group = []
                        auc_scores_for_group = []
                        precision_scores_for_group = []
                        recall_scores_for_group = []
                        seed_count = 0
                        for seed in ds_split_seeds:
                            print(method, count, 'out of', len(gnn_params['lr'])*len(gnn_params['w'])*len(gnn_params['K'])*len(gnn_params['F'])*len(ds_split_seeds))


                            ds = mh.Dataset()
                            ds.load_dataset(folder=dataset_folder+ds_name,splits=[0.5,0.2,0.3],split_type="normal",split_seed=seed)

                            model = mh.Model(ds.data, gridsearch_flag=True)
                            model.w = w
                            model.lr = lr
                            model.load_model(method,K=K,F=F)
                            model.best_f1_from_gs = gnn_params['mean_f1']
                            #model.train_model(epochs=gnn_params['epochs'][seed_count])
                            model.train_model(epochs=5000)
                            f1_scores_for_group.append(model.gridsearch_results['test_f1'])
                            best_epochs_for_group.append(model.gridsearch_results['test_epoch'])
                            auc_scores_for_group.append(model.gridsearch_results['test_auc'])
                            precision_scores_for_group.append(model.gridsearch_results['test_precision'])
                            recall_scores_for_group.append(model.gridsearch_results['test_recall'])
                            count+=1
                            seed_count+=1
                        '''
                        if np.abs(model.gridsearch_results['test_f1'] - gnn_params['mean_f1']) < best_model_diff:
                            best_model_diff = np.abs(model.gridsearch_results['test_f1'] - gnn_params['mean_f1'])
                            closest_f1 = model.gridsearch_results['test_f1']
                            tpfp = model.check_topology_performance(folder=ds.folder, data=ds.data)
                            results = {
                            'tp':tpfp[0],
                            'fp':tpfp[1],
                            }
                        '''
                        '''      
                        for ele in results:
                            gs_results[ds_name][method][ele] = results[ele]
                            print(ele,results[ele])
                        with open(results_path, 'w') as file:
                            json.dump(gs_results, file, indent=2)
                        '''       
                        
    #print("closest to mean f1:",closest_f1)
    #print("original median f1:", gnn_params['mean_f1'])
    #print("diff f1:", np.abs(closest_f1-gnn_params['mean_f1']))
    #print("results:",results)
    return results

In [12]:
def rf_toposearch(ds,param_grid):
    '''
    param_grid = {'bootstrap': [True],
        'max_depth': [70],
        'max_features': ['sqrt'],
        'min_samples_leaf': [4],
        'min_samples_split': [5],
        'n_estimators': [50]}
    '''
    trainX = ds.data.x[ds.data.train_mask + ds.data.val_mask].detach().cpu().numpy()
    trainY = ds.data.y[ds.data.train_mask + ds.data.val_mask].detach().cpu().numpy()

    testX = ds.data.x[ds.data.test_mask].detach().cpu().numpy()
    testY = ds.data.y[ds.data.test_mask].detach().cpu().numpy()

    clf = RandomForestClassifier()
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3, n_jobs=-1)
    grid_search.fit(trainX, trainY)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_

    #clf = RandomForestClassifier(bootstrap=best_params['bootstrap'],max_depth=best_params['max_depth'],max_features=best_params['max_features'],min_samples_leaf=best_params['min_samples_leaf'],min_samples_split=best_params['min_samples_split'],n_estimators=best_params['n_estimators'])
    #st = time.time()
    #clf.fit(trainX, trainY)
    #et = time.time()
    #elapsed_time = et - st
    #best_model = clf
    
    test_pred = best_model.predict(testX)
    y_pred_prob = best_model.predict_proba(testX)
    y_pred_prob = y_pred_prob[:,1]

    precision, recall, f1_score, _ = precision_recall_fscore_support(testY, test_pred, average='binary')                     
    fpr, tpr, thresholds = roc_curve(testY, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    #test_acc, test_out, test_pred = test(ds.data.test_mask,ds.data)
    #test_pred = test_pred.detach().cpu()

    test_pred = np.round(y_pred_prob)
    y_true = ds.data.y[ds.data.test_mask].cpu()
    precision, recall, f1_score, _ = precision_recall_fscore_support(ds.data.y[ds.data.test_mask].cpu(), test_pred, average='binary')
    folder = "v2/128K_05_v2_2"
    accounts_df = pd.read_csv("../datasets/"+folder+"/account_attributes.csv")

    indicies = accounts_df.index

    out = np.array(test_pred)  # Convert 'out' to a NumPy array for easier indexing
    true_labels = np.array(y_true)  # Convert 'true_labels' to a NumPy array for easier indexing

    # True Positives (TP)
    tp_indices = np.where((out == 1) & (true_labels == 1))[0]
    print("true positives =",len(tp_indices))
    tp_indices = indicies[ds.data.test_mask.cpu().numpy()][tp_indices]

    # True Negatives (TN)
    tn_indices = np.where((out == 0) & (true_labels == 0))[0]
    print("true negatives =",len(tn_indices))
    tn_indices = indicies[ds.data.test_mask.cpu().numpy()][tn_indices]

    # False Positives (FP)
    fp_indices = np.where((out == 1) & (true_labels == 0))[0]
    print("false positives =",len(fp_indices))
    fp_indices = indicies[ds.data.test_mask.cpu().numpy()][fp_indices]

    # False Negatives (FN)
    fn_indices = np.where((out == 0) & (true_labels == 1))[0]
    print("false negatives =",len(fn_indices))
    fn_indices = indicies[ds.data.test_mask.cpu().numpy()][fn_indices]

    '''
    accounts_df['trained_color'] = "#000000"
    accounts_df['trained_color'][train_mask.numpy()] = "#343B46"
    accounts_df['trained_color'][val_mask.numpy()] = "#343B46"
    accounts_df['trained_color'][tp_indices] = "#eb4034"
    accounts_df['trained_color'][tn_indices] = "#1ad623"
    accounts_df['trained_color'][fp_indices] = "#26e0be"
    accounts_df['trained_color'][fn_indices] = "#fae01e"
    '''

    accounts_df['trained_color'] = 1
    accounts_df['trained_color'][ds.data.train_mask.cpu().numpy()] = 2
    accounts_df['trained_color'][ds.data.val_mask.cpu().numpy()] = 2
    accounts_df['trained_color'][tp_indices] = 3
    accounts_df['trained_color'][tn_indices] = 4
    accounts_df['trained_color'][fp_indices] = 5
    accounts_df['trained_color'][fn_indices] = 6

    #accounts_df['id'] = indicies
    accounts_df['id'] = (accounts_df['id']).astype(int)

    #accounts_df.to_csv('../datasets/60K_01_sar_count/account_attributes_trained_vis_SAGE.csv',index=0)

    #np.sum(accounts_df['trained_color'] == 0), len(train_dataset), len(val_dataset), len(test_dataset)

    sar_df = pd.read_csv("../datasets/"+folder+"/alert_accounts.csv")

    ids_cycle = sar_df["acct_id"][sar_df["alert_type"]=='cycle'].to_numpy() #super important to convert to numpy array!!!!! keeping as pandas series messes up all indexing!!!
    ids_fan_in = sar_df["acct_id"][sar_df["alert_type"]=='fan_in'].to_numpy()
    ids_fan_out = sar_df["acct_id"][sar_df["alert_type"]=='fan_out'].to_numpy()
    ids_gather_scatter = sar_df["acct_id"][sar_df["alert_type"]=='gather_scatter'].to_numpy()
    ids_scatter_gather = sar_df["acct_id"][sar_df["alert_type"]=='scatter_gather'].to_numpy()
    ids_bipartite = sar_df["acct_id"][sar_df["alert_type"]=='bipartite'].to_numpy()
    ids_stack = sar_df["acct_id"][sar_df["alert_type"]=='stack'].to_numpy()
    set(sar_df["alert_type"])

    #print('ids_cycle',ids_cycle)

    ids_test_set = accounts_df['id'][ds.data.test_mask.cpu().numpy()].astype(int).to_numpy()
    labels_test_set = ds.data.y[ds.data.test_mask].cpu().numpy()
    predictions_test_set = test_pred
    correct_pos_pred = np.logical_and(labels_test_set, predictions_test_set) # is can only equal 1 if both label and precition were 1
    print('correct_pos_pred',correct_pos_pred)
    print("sanity check: num true positives, num total positives, sanity check, recall")
    print(np.sum(correct_pos_pred), np.sum(labels_test_set), np.round(np.sum(correct_pos_pred)/np.sum(labels_test_set),4), np.round(recall,4))
    print(len(correct_pos_pred), len(ids_test_set),len(sar_df["acct_id"]))
    print(len(labels_test_set),len(predictions_test_set))

    print(type(ids_test_set))

    pred_cycle = correct_pos_pred[[e in ids_cycle for e in ids_test_set]]
    pred_fan_in = correct_pos_pred[[e in ids_fan_in for e in ids_test_set]]
    pred_fan_out = correct_pos_pred[[e in ids_fan_out for e in ids_test_set]]
    pred_gather_scatter = correct_pos_pred[[e in ids_gather_scatter for e in ids_test_set]]
    pred_scatter_gather = correct_pos_pred[[e in ids_scatter_gather for e in ids_test_set]]
    pred_bipartite = correct_pos_pred[[e in ids_bipartite for e in ids_test_set]]
    pred_stack = correct_pos_pred[[e in ids_stack for e in ids_test_set]]
    print('pred cycle',pred_cycle)

    #sanity check
    print('sanity check')
    print((len(pred_cycle) + len(pred_fan_in) + len(pred_fan_out) + len(pred_gather_scatter) + len(pred_scatter_gather)), np.sum(np.array(labels_test_set)), np.sum(np.array(correct_pos_pred)), len(correct_pos_pred))

    topologies = ('Cycle', 'Fan In', 'Fan Out', 'Gather Scatter', 'Scatter Gather', 'Bipartite', 'Stack')

    tp = np.round(np.array([np.sum(pred_cycle)/len(pred_cycle), np.sum(pred_fan_in)/len(pred_fan_in), np.sum(pred_fan_out)/len(pred_fan_out), np.sum(pred_gather_scatter)/len(pred_gather_scatter), np.sum(pred_scatter_gather)/len(pred_scatter_gather), np.sum(pred_bipartite)/len(pred_bipartite), np.sum(pred_stack)/len(pred_stack)]),2)
    fp = np.ones(len(tp)) - tp
    #print(tp)
    topology_counts = {
        'True Positive': tp,
        'False Positive': fp,
    }
    print(f1_score)

    #return results

In [13]:
def xg_toposearch(ds,params):
    trainX = ds.data.x[ds.data.train_mask + ds.data.val_mask].detach().cpu().numpy()
    trainY = ds.data.y[ds.data.train_mask + ds.data.val_mask].detach().cpu().numpy()

    testX = ds.data.x[ds.data.test_mask].detach().cpu().numpy()
    testY = ds.data.y[ds.data.test_mask].detach().cpu().numpy()

    clf = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
    '''
    params = {
        "gamma": [0],
        'max_depth': [30],
        'n_estimators': [100]}
    '''
    grid_search = GridSearchCV(estimator=clf, param_grid=params, cv=3, n_jobs=-1)
    grid_search.fit(trainX, trainY)

    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    
    #clf = xgb.XGBClassifier(objective="binary:logistic", random_state=42,gamma=best_params["gamma"], max_depth=best_params["max_depth"],n_estimators=best_params["n_estimators"])
    #st = time.time()
    #clf.fit(trainX, trainY)
    #et = time.time()
    #elapsed_time = et - st

    test_pred = best_model.predict(testX)
    test_pred = np.round(test_pred)
    print(np.sum(test_pred))

    y_pred_prob = best_model.predict_proba(testX)
    y_pred_prob = y_pred_prob[:,1]

    precision, recall, f1_score, _ = precision_recall_fscore_support(testY, test_pred, average='binary')                     
    fpr, tpr, thresholds = roc_curve(testY, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    #test_acc, test_out, test_pred = test(ds.data.test_mask,ds.data)
    #test_pred = test_pred.detach().cpu()

    test_pred = np.round(y_pred_prob)
    y_true = ds.data.y[ds.data.test_mask].cpu()
    precision, recall, f1_score, _ = precision_recall_fscore_support(ds.data.y[ds.data.test_mask].cpu(), test_pred, average='binary')
    folder = "v2/128K_05_v2_2"
    accounts_df = pd.read_csv("../datasets/"+folder+"/account_attributes.csv")

    indicies = accounts_df.index

    out = np.array(test_pred)  # Convert 'out' to a NumPy array for easier indexing
    true_labels = np.array(y_true)  # Convert 'true_labels' to a NumPy array for easier indexing

    # True Positives (TP)
    tp_indices = np.where((out == 1) & (true_labels == 1))[0]
    print("true positives =",len(tp_indices))
    tp_indices = indicies[ds.data.test_mask.cpu().numpy()][tp_indices]

    # True Negatives (TN)
    tn_indices = np.where((out == 0) & (true_labels == 0))[0]
    print("true negatives =",len(tn_indices))
    tn_indices = indicies[ds.data.test_mask.cpu().numpy()][tn_indices]

    # False Positives (FP)
    fp_indices = np.where((out == 1) & (true_labels == 0))[0]
    print("false positives =",len(fp_indices))
    fp_indices = indicies[ds.data.test_mask.cpu().numpy()][fp_indices]

    # False Negatives (FN)
    fn_indices = np.where((out == 0) & (true_labels == 1))[0]
    print("false negatives =",len(fn_indices))
    fn_indices = indicies[ds.data.test_mask.cpu().numpy()][fn_indices]

    '''
    accounts_df['trained_color'] = "#000000"
    accounts_df['trained_color'][train_mask.numpy()] = "#343B46"
    accounts_df['trained_color'][val_mask.numpy()] = "#343B46"
    accounts_df['trained_color'][tp_indices] = "#eb4034"
    accounts_df['trained_color'][tn_indices] = "#1ad623"
    accounts_df['trained_color'][fp_indices] = "#26e0be"
    accounts_df['trained_color'][fn_indices] = "#fae01e"
    '''

    accounts_df['trained_color'] = 1
    accounts_df['trained_color'][ds.data.train_mask.cpu().numpy()] = 2
    accounts_df['trained_color'][ds.data.val_mask.cpu().numpy()] = 2
    accounts_df['trained_color'][tp_indices] = 3
    accounts_df['trained_color'][tn_indices] = 4
    accounts_df['trained_color'][fp_indices] = 5
    accounts_df['trained_color'][fn_indices] = 6

    #accounts_df['id'] = indicies
    accounts_df['id'] = (accounts_df['id']).astype(int)

    #accounts_df.to_csv('../datasets/60K_01_sar_count/account_attributes_trained_vis_SAGE.csv',index=0)

    #np.sum(accounts_df['trained_color'] == 0), len(train_dataset), len(val_dataset), len(test_dataset)

    sar_df = pd.read_csv("../datasets/"+folder+"/alert_accounts.csv")

    ids_cycle = sar_df["acct_id"][sar_df["alert_type"]=='cycle'].to_numpy() #super important to convert to numpy array!!!!! keeping as pandas series messes up all indexing!!!
    ids_fan_in = sar_df["acct_id"][sar_df["alert_type"]=='fan_in'].to_numpy()
    ids_fan_out = sar_df["acct_id"][sar_df["alert_type"]=='fan_out'].to_numpy()
    ids_gather_scatter = sar_df["acct_id"][sar_df["alert_type"]=='gather_scatter'].to_numpy()
    ids_scatter_gather = sar_df["acct_id"][sar_df["alert_type"]=='scatter_gather'].to_numpy()
    ids_bipartite = sar_df["acct_id"][sar_df["alert_type"]=='bipartite'].to_numpy()
    ids_stack = sar_df["acct_id"][sar_df["alert_type"]=='stack'].to_numpy()
    set(sar_df["alert_type"])

    #print('ids_cycle',ids_cycle)

    ids_test_set = accounts_df['id'][ds.data.test_mask.cpu().numpy()].astype(int).to_numpy()
    labels_test_set = ds.data.y[ds.data.test_mask].cpu().numpy()
    predictions_test_set = test_pred
    correct_pos_pred = np.logical_and(labels_test_set, predictions_test_set) # is can only equal 1 if both label and precition were 1
    print('correct_pos_pred',correct_pos_pred)
    print("sanity check: num true positives, num total positives, sanity check, recall")
    print(np.sum(correct_pos_pred), np.sum(labels_test_set), np.round(np.sum(correct_pos_pred)/np.sum(labels_test_set),4), np.round(recall,4))
    print(len(correct_pos_pred), len(ids_test_set),len(sar_df["acct_id"]))
    print(len(labels_test_set),len(predictions_test_set))

    print(type(ids_test_set))

    pred_cycle = correct_pos_pred[[e in ids_cycle for e in ids_test_set]]
    pred_fan_in = correct_pos_pred[[e in ids_fan_in for e in ids_test_set]]
    pred_fan_out = correct_pos_pred[[e in ids_fan_out for e in ids_test_set]]
    pred_gather_scatter = correct_pos_pred[[e in ids_gather_scatter for e in ids_test_set]]
    pred_scatter_gather = correct_pos_pred[[e in ids_scatter_gather for e in ids_test_set]]
    pred_bipartite = correct_pos_pred[[e in ids_bipartite for e in ids_test_set]]
    pred_stack = correct_pos_pred[[e in ids_stack for e in ids_test_set]]
    print('pred cycle',pred_cycle)

    #sanity check
    print('sanity check')
    print((len(pred_cycle) + len(pred_fan_in) + len(pred_fan_out) + len(pred_gather_scatter) + len(pred_scatter_gather)), np.sum(np.array(labels_test_set)), np.sum(np.array(correct_pos_pred)), len(correct_pos_pred))

    topologies = ('Cycle', 'Fan In', 'Fan Out', 'Gather Scatter', 'Scatter Gather', 'Bipartite', 'Stack')

    tp = np.round(np.array([np.sum(pred_cycle)/len(pred_cycle), np.sum(pred_fan_in)/len(pred_fan_in), np.sum(pred_fan_out)/len(pred_fan_out), np.sum(pred_gather_scatter)/len(pred_gather_scatter), np.sum(pred_scatter_gather)/len(pred_scatter_gather), np.sum(pred_bipartite)/len(pred_bipartite), np.sum(pred_stack)/len(pred_stack)]),2)
    fp = np.ones(len(tp)) - tp
    #print(tp)
    topology_counts = {
        'True Positive': tp,
        'False Positive': fp,
    }
    print(f1_score)

    #return results

In [17]:
#loop format
#loop through datasets
#loop through models
#gridsearch model

dataset_folder = 'v2/'
#dataset_folder = 'thesis_datasets/'
#datasets = ['128K_05_v2_2','128K_1_v2_2','128K_5_v2_2','128K_10_v2_2']
#datasets = ['128K_5_v2_2','128K_10_v2_2','128K_1_v2_2','128K_05_v2_2']
datasets = ['64K_5_v2']
#datasets = ['128K_5_v2_2']
methods = ['SAGE','XG','RF']
methods = ['GINSAGE','SAGE','XG','RF']
#methods = ['SAGE','GINSAGE']
methods = ['XG','RF']
#methods = ['RF','XG']

now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
current_time = now.strftime("%H-%M-%S")
result_string = f"{current_date}_{current_time}"

results_path = 'gridsearch_results.json'
#results_path = 'gridsearch_results/gridsearch_results_'+result_string+'.json'


gs_results = {}
try:
    # Try to open the existing JSON file
    with open(results_path, 'r') as file:
        gs_results = json.load(file)
        print("Existing JSON gridsearch results file loaded.")
except FileNotFoundError:
    # If the file doesn't exist, create an empty dictionary
    #gs_results = {}
    print("JSON gridsearch results file not found")
    #with open(results_path, 'w') as file:
    #    json.dump(gs_results, file, indent=2)

for ds_name in datasets:

    split_seeds = [0,111,222]
    #split_seeds = [0,111,222,333,444,555,666,777,888,999]
    #split_seeds = [0]
    ds = mh.Dataset()
    ds.load_dataset(folder=dataset_folder+ds_name,splits=[0.5,0.2,0.3],split_type="normal")
    print('Running Dataset',ds_name)

    for method in methods:
        model_path = './best_models/'+dataset_folder+ds_name+'_'+method+'.pth'
        print('    Running method',method)
       

        
        #gs_results[ds_name][method] = {}
        
        
        results = {}
        if method in ['GIN','SAGE','MPNN','GCN','GAT','GINSAGE']:
            gnn_params = {
                'lr':[gs_results[ds_name][method]['lr']],
                'w':[gs_results[ds_name][method]['w']],
                'K':[gs_results[ds_name][method]['K']],
                'F':[gs_results[ds_name][method]['F']],
                'K1':[gs_results[ds_name][method]['K1']],
                'K2':[gs_results[ds_name][method]['K2']],
                'F1':[gs_results[ds_name][method]['F1']],
                'F2':[gs_results[ds_name][method]['F2']],
                'mean_f1':gs_results[ds_name][method]['mean_f1'],
                'epochs':gs_results[ds_name][method]['epoch']
            }
            model = mh.Model(ds.data, gridsearch_flag=True)
            results = gnn_toposearch(split_seeds, dataset_folder, method, gnn_params, ds_name, results_path, gs_results)
            '''
            try:
                results = gnn_gridsearch(model, ds, method, gnn_params)
            except:
                print(method,'on dataset',ds_name,'has crashed. Continuing to next seach.')
            '''
        elif method == 'XG':
            xg_params = {
                "gamma": [gs_results[ds_name][method]['gamma']],
                'max_depth': [gs_results[ds_name][method]['max_depth']],
                'n_estimators': [gs_results[ds_name][method]['n_estimators']]}
            for seed in split_seeds:
                print("seed",seed)
                ds = mh.Dataset()
                ds.load_dataset(folder=dataset_folder+ds_name,splits=[0.5,0.2,0.3],split_type="normal",split_seed=seed)
                results = xg_toposearch(ds, xg_params)
        elif method == 'RF':
            rf_params = {'bootstrap': [gs_results[ds_name][method]['bootstrap']],
                'max_depth': [gs_results[ds_name][method]['max_depth']],
                'max_features': [gs_results[ds_name][method]['max_features']],
                'min_samples_leaf': [gs_results[ds_name][method]['min_samples_leaf']],
                'min_samples_split': [gs_results[ds_name][method]['min_samples_split']],
                'n_estimators': [gs_results[ds_name][method]['n_estimators']]}
            for seed in split_seeds:
                print("seed",seed)
                ds = mh.Dataset()
                ds.load_dataset(folder=dataset_folder+ds_name,splits=[0.5,0.2,0.3],split_type="normal",split_seed=seed)
                results = rf_toposearch(ds,rf_params)
        else:
            print('method',method,'does not exist!')
        '''
        for ele in results:
            gs_results[ds_name][method][ele] = results[ele]
            print(ele,results[ele])

        with open(results_path, 'w') as file:
            json.dump(gs_results, file, indent=2)
        '''







Existing JSON gridsearch results file loaded.
device: cuda:0
loading dataset v2/64K_5_v2 | length: 64026 | fraud percentage (%): 4.91
Running Dataset 64K_5_v2
    Running method XG
seed 0
device: cuda:0
loading dataset v2/64K_5_v2 | length: 64026 | fraud percentage (%): 4.91
682
true positives = 660


IndexError: boolean index did not match indexed array along dimension 0; dimension is 128002 but corresponding boolean dimension is 64026